In [1]:
import numpy as np
import pandas as pd
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
import pickle
import matplotlib.pyplot as plt
import chess
import random
from utils.data_loader import load_training_data, encode_board_state, decode_prediction
from utils.visualization import live_visualize_game


ModuleNotFoundError: No module named 'chess'

In [ ]:
def plot_training_results(train_scores, val_scores, neighbors_range):
    """Plot training and validation accuracy over different k values."""
    plt.figure(figsize=(10, 6))
    plt.plot(neighbors_range, train_scores, label="Training Accuracy", marker="o")
    plt.plot(neighbors_range, val_scores, label="Validation Accuracy", marker="o")
    plt.xlabel("Number of Neighbors (k)")
    plt.ylabel("Accuracy")
    plt.title("Training and Validation Accuracy vs. k")
    plt.legend()
    plt.grid()
    plt.show()

In [ ]:
def train_knn_model(data_path, model_save_path, neighbors_range=range(1, 11)):
    """Train a KNN model and evaluate for a range of k values."""
    # Load training data
    X, y = load_training_data(data_path)
    X = X.reshape(X.shape[0], -1)  # Flatten board states for KNN

    # Split data into training and validation sets
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

    train_scores = []
    val_scores = []

    # Iterate over a range of k values to find the best one
    for k in neighbors_range:
        knn = KNeighborsClassifier(n_neighbors=k)
        knn.fit(X_train, y_train)

        train_scores.append(knn.score(X_train, y_train))
        val_scores.append(knn.score(X_val, y_val))

    # Plot results
    plot_training_results(train_scores, val_scores, neighbors_range)

    # Select the best k value based on validation score
    best_k = neighbors_range[np.argmax(val_scores)]
    print(f"Best k: {best_k} with Validation Accuracy: {max(val_scores):.2f}")

    # Train final model with the best k
    best_knn = KNeighborsClassifier(n_neighbors=best_k)
    best_knn.fit(X_train, y_train)

    # Save the final model
    with open(model_save_path, 'wb') as file:
        pickle.dump(best_knn, file)

    print(f"Model saved to {model_save_path}")

In [ ]:
def play_self_game(model_path):
    """Play a chess game where the AI plays against itself."""
    with open(model_path, 'rb') as file:
        knn_model = pickle.load(file)

    board = chess.Board()
    while not board.is_game_over():
        if board.turn:  # AI's turn (White)
            encoded_board = encode_board_state(board).reshape(1, -1)
            predictions = knn_model.predict(encoded_board)
            best_move = decode_prediction(predictions, board)
            board.push(best_move)
        else:  # AI's turn (Black)
            encoded_board = encode_board_state(board).reshape(1, -1)
            predictions = knn_model.predict(encoded_board)
            best_move = decode_prediction(predictions, board)
            board.push(best_move)

    print("Self-Play Game Over:", board.result())


In [ ]:
def play_stockfish_game(model_path):
    """Play a chess game where the AI plays against Stockfish."""
    from stockfish import Stockfish
    stockfish = Stockfish("stockfish/stockfish")

    with open(model_path, 'rb') as file:
        knn_model = pickle.load(file)

    board = chess.Board()
    while not board.is_game_over():
        if board.turn:  # AI's turn (White)
            encoded_board = encode_board_state(board).reshape(1, -1)
            predictions = knn_model.predict(encoded_board)
            best_move = decode_prediction(predictions, board)
            board.push(best_move)
        else:  # Stockfish's turn (Black)
            stockfish.set_fen_position(board.fen())
            stockfish_move = stockfish.get_best_move()
            board.push(chess.Move.from_uci(stockfish_move))

    print("Game Against Stockfish Over:", board.result())

Specify file paths and parameters

In [ ]:
data_path = "data/training_games.csv"
model_save_path = "data/models/knn_model.pkl"

Continuous Training with Self-Play and Stockfish Games

In [ ]:
continuous_training_with_games(data_path, model_save_path)
